<a href="https://colab.research.google.com/github/tkarab/diplomaThesis/blob/feat%2F76-feat-siamese-network-callbacks/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # 1 - REQUIREMENTS


In [1]:
# @title Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# @title Install tensorflow older version
!pip install tensorflow==2.10.0


In [3]:
# @title Imports
import numpy as np
import pandas as pd
import csv
import json
from keras import utils, initializers, regularizers
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
print(tf.__version__)
import keras
from keras import layers
from keras.layers import Lambda, TimeDistributed
from keras import callbacks as cb
import tensorflow.keras.backend as K

import os
import sys
import re
import time
from tqdm.notebook import tqdm

import random
import scipy
from matplotlib import pyplot as plt
from scipy.fft import fft, ifft




2.10.0


In [4]:
# @title Paths
ROOT = '/content/drive/MyDrive'

#My google Drive
drivePath = '/content/drive/MyDrive'

#Colab Notebooks
colabNotebooksPath = drivePath+'/Colab Notebooks'
modulesPath = colabNotebooksPath + '/Modules'
nb_path = '/content/notebooks'  #Εκεί βρίσκονται τα εκτελέσιμα και οι βιβλιοθήκες


#Python files
sourcefilePath = drivePath+'/Source Files'

#Data files
dataPath = drivePath + '/Data'
ninaproPath = dataPath + '/Ninapro'
separatedDataPath = dataPath + '/Separated'
db2SeparatedPath = os.path.join(separatedDataPath,'db2.npz')
preprocessedDataPath = dataPath + '/Preprocessed'
db2ProcessedPath = os.path.join(preprocessedDataPath,'DB2')

#Data
DATA_PATH = ROOT + r'/Data'
NINAPRO_PATH = ROOT + r'/Data/Ninapro'
SEPARATED_DATA_PATH = os.path.join(DATA_PATH ,'Separated')

PROCESSED_DATA_PATH_DB2 = ROOT + r'/Data/Preprocessed/DB2'
PROCESSED_DATA_PATH_DB1 = ROOT + r'/Data/Preprocessed/DB1'
PROCESSED_DATA_PATH_DB5 = ROOT + r'/Data/Preprocessed/DB5'

RMS_DATA_PATH_DB1 = os.path.join(PROCESSED_DATA_PATH_DB1,'rms')
RMS_DATA_PATH_DB2 = os.path.join(PROCESSED_DATA_PATH_DB2,'rms')
RMS_DATA_PATH_DB5 = os.path.join(PROCESSED_DATA_PATH_DB5,'rms')

#Tasks
TASKS_FILE_PATH_DB1 = ROOT + r'/Data/Tasks/DB1'
TASKS_FILE_PATH_DB2 = ROOT + r'/Data/Tasks/DB2'
TASKS_FILE_PATH_DB5 = ROOT + r'/Data/Tasks/DB5'

#Results
RESULTS_PATH_EX1  = ROOT + r'/Data/Results/Experiment 1'
RESULTS_PATH_EX2A = ROOT + r'/Data/Results/Experiment 2a'
RESULTS_PATH_EX2B = ROOT + r'/Data/Results/Experiment 2b'
RESULTS_PATH_EX3  = ROOT + r'/Data/Results/Experiment 3'
RESULTS_DIRECTORIES_DICT = {
    '1' : RESULTS_PATH_EX1,
    '2a': RESULTS_PATH_EX2A,
    '2b': RESULTS_PATH_EX2B,
    '3' : RESULTS_PATH_EX3
}

#Config
DATA_CONFIG_PATH_PREPROC = ROOT + r'/Data/Config/preproc'
DATA_CONFIG_PATH_AUG = ROOT + r'/Data/Config/aug'

#Images
imagesPath = drivePath + '/Images'
imagesPreprocessingPath = imagesPath + '/Preprocessing'

# 2 - MODELS

In [5]:
# @title Custom models

def AtzoriNetDB2_embedding_only(input_shape = (15,12,1), add_dropout = False, dropout_pct = 0.15, krnl_init_name = "glorot_normal", add_regularizer:bool = False, l2 = 0.0002):
    # Kernel Initializer
    if krnl_init_name == "glorot_normal":
        kernel_init = initializers.glorot_normal(seed=0)
    else:
        kernel_init = initializers.glorot_uniform(seed=0)

    if add_regularizer:
        kernel_reg = regularizers.l2(l2)
    else:
        kernel_reg = None

    # Input
    X_inp = layers.Input(shape=input_shape)

    # Layer 1: Padding (0,5) -> Conv2D [32 x (1,12)] -> ReLU
    # Input:  (15,12,1)
    # Output: (15,11,32)
    # X = layers.BatchNormalization(X_inp)
    X = layers.BatchNormalization()(X_inp)
    X = layers.ZeroPadding2D(padding=(0, 5))(X)
    X = layers.Conv2D(filters=32, kernel_size=(1, 12), padding='valid', activation='relu', kernel_initializer=kernel_init, kernel_regularizer=kernel_reg)(X)
    if add_dropout == True:
        X = layers.Dropout(dropout_pct)(X)

    # Layer 2: Padding (1,1) -> Conv2D [32 x (3,3)] -> ReLU -> AvgPooling(3,3)
    # Input:  (15,11,32)
    # Output: (5,3,32)
    X = layers.BatchNormalization()(X)
    X = layers.ZeroPadding2D(padding=(1, 1))(X)
    X = layers.Conv2D(filters=32, kernel_size=(3, 3), padding='valid', activation='relu', kernel_initializer=kernel_init, kernel_regularizer=kernel_reg)(X)
    if add_dropout == True:
        X = layers.Dropout(dropout_pct)(X)
    X = layers.AveragePooling2D(pool_size=(3, 3))(X)

    # Layer 3: Padding (2,2) -> Conv2D [64 x (5,5)] -> ReLU -> AvgPooling(3,3)
    # Input:  (5,3,32)
    # Output: (1,1,64)
    X = layers.BatchNormalization()(X)
    X = layers.ZeroPadding2D(padding=(2, 2))(X)
    X = layers.Conv2D(filters=64, kernel_size=(5, 5), padding='valid', activation='relu', kernel_initializer=kernel_init, kernel_regularizer=kernel_reg)(X)
    if add_dropout == True:
        X = layers.Dropout(dropout_pct)(X)
    X = layers.AveragePooling2D(pool_size=(3, 3))(X)

    # Layer 4: Padding (4,0) -> Conv2D [64 x (9,1)] -> ReLU
    # The Output is a vector of length 64 corresponding to the input image
    # Input:  (1,1,64)
    # Output: (1,1,64)
    X = layers.BatchNormalization()(X)
    X = layers.ZeroPadding2D(padding=(4, 0))(X)
    X = layers.Conv2D(filters=64, kernel_size=(9, 1), padding='valid', activation='relu', kernel_initializer=kernel_init, kernel_regularizer=kernel_reg)(X)
    if add_dropout == True:
        X = layers.Dropout(dropout_pct)(X)

    Y = layers.Flatten()(X)

    model = keras.Model(name = 'AtzoriNetDB2',
                        inputs = X_inp,
                        outputs = Y)

    return model

def AtzoriNetDB2_embedding_only_extra_layers_added(input_shape = (15,12,1), extra_layers=1, add_dropout = False, dropout_pct = 0.15, krnl_init_name = "glorot_normal", add_regularizer:bool = False, l2 = 0.0002):
    # Kernel Initializer
    if krnl_init_name == "glorot_normal":
        kernel_init = initializers.glorot_normal(seed=0)
    else:
        kernel_init = initializers.glorot_uniform(seed=0)

    if add_regularizer:
        kernel_reg = regularizers.l2(l2)
    else:
        kernel_reg = None

    # Input
    X_inp = layers.Input(shape=input_shape)

    # Layer 1: Padding (0,5) -> Conv2D [32 x (1,12)] -> ReLU
    # Input:  (15,12,1)
    # Output: (15,11,32)
    # X = layers.BatchNormalization(X_inp)
    X = layers.BatchNormalization()(X_inp)
    X = layers.ZeroPadding2D(padding=(0, 5))(X)
    X = layers.Conv2D(filters=32, kernel_size=(1, 12), padding='valid', activation='relu', kernel_initializer=kernel_init, kernel_regularizer=kernel_reg)(X)
    if add_dropout == True:
        X = layers.Dropout(dropout_pct)(X)

    # Layer 2: Padding (1,1) -> Conv2D [32 x (3,3)] -> ReLU -> AvgPooling(3,3)
    # Input:  (15,11,32)
    # Output: (5,3,32)
    X = layers.BatchNormalization()(X)
    X = layers.ZeroPadding2D(padding=(1, 1))(X)
    X = layers.Conv2D(filters=32, kernel_size=(3, 3), padding='valid', activation='relu', kernel_initializer=kernel_init, kernel_regularizer=kernel_reg)(X)
    if add_dropout == True:
        X = layers.Dropout(dropout_pct)(X)
    X = layers.AveragePooling2D(pool_size=(3, 3))(X)

    # Extra layers
    # Adding extra layers of 32 (3,3) filter with no Pooling
    # Therefore the shape of the data does not change
    # Input:  (5,3,32)
    # Output: (5,3,32)
    for i in range(extra_layers):
        X = layers.BatchNormalization()(X)
        X = layers.ZeroPadding2D(padding=(1, 1))(X)
        X = layers.Conv2D(filters=32, kernel_size=(3, 3), padding='valid', activation='relu',
                          kernel_initializer=kernel_init, kernel_regularizer=kernel_reg)(X)
        if add_dropout == True:
            X = layers.Dropout(dropout_pct)(X)


    # Layer 3: Padding (2,2) -> Conv2D [64 x (5,5)] -> ReLU -> AvgPooling(3,3)
    # Input:  (5,3,32)
    # Output: (1,1,64)
    X = layers.BatchNormalization()(X)
    X = layers.ZeroPadding2D(padding=(2, 2))(X)
    X = layers.Conv2D(filters=64, kernel_size=(5, 5), padding='valid', activation='relu', kernel_initializer=kernel_init, kernel_regularizer=kernel_reg)(X)
    if add_dropout == True:
        X = layers.Dropout(dropout_pct)(X)
    X = layers.AveragePooling2D(pool_size=(3, 3))(X)

    # Layer 4: Padding (4,0) -> Conv2D [64 x (9,1)] -> ReLU
    # The Output is a vector of length 64 corresponding to the input image
    # Input:  (1,1,64)
    # Output: (1,1,64)
    X = layers.BatchNormalization()(X)
    X = layers.ZeroPadding2D(padding=(4, 0))(X)
    X = layers.Conv2D(filters=64, kernel_size=(9, 1), padding='valid', activation='relu', kernel_initializer=kernel_init, kernel_regularizer=kernel_reg)(X)
    if add_dropout == True:
        X = layers.Dropout(dropout_pct)(X)

    Y = layers.Flatten()(X)
    name = f'AtzoriNetDB2_{extra_layers}ExtraLayer'
    if extra_layers > 1:
        name += 's'

    model = keras.Model(name = name,
                        inputs = X_inp,
                        outputs = Y)

    return model



In [6]:
# @title fsl functions

def produce_prototype(x):
    return tf.reduce_mean(x,axis=-2)

def reshape_query(x):
    return tf.reshape(x,[-1,tf.shape(x)[-1]])

def keep_first_tensor(x):
    print_tensor_shape(x)
    x = tf.expand_dims(x[0],axis=0)
    print_tensor_shape(x)

    return x

def get_sum_of_squares(x):
    return tf.reduce_sum(x**2,axis=1,keepdims=True)

def euc_dist(args):
    prototypes, query_feat = args

    return tf.sqrt(tf.reduce_sum(tf.square(prototypes-query_feat), axis=-1))

def l2_dist(args):
    x1, x2 = args
    return tf.square(x1-x2)

def l1_dist(args):
    x1, x2 = args
    return tf.abs(x1-x2)

def inner_product(args):
    x1, x2 = args
    return x1*x2

def inner_product_norm(args):
    x1, x2 = args
    return (x1/tf.norm(x1))*(x2/tf.norm(x2))

def inner_product_abs(args):
    x1, x2 = args
    return tf.abs(x1*x2)

def inner_product_norm_abs(args):
    x1, x2 = args
    return tf.abs((x1/tf.norm(x1))*(x2/tf.norm(x2)))


def softmax_classification(args, print_result=False):
    pred = tf.nn.softmax(-euc_dist(args),axis=-1)
    if print_result:
        print_tensor(pred)
    return pred

def get_fsl_set_rand(N,k,dim1=28,dim2=28):
    x = tf.random.uniform(shape=(N, k, dim1, dim2, 1), minval=0, maxval=1)
    return x

def get_fsl_set(x,N,k):
    return tf.stack([tf.concat([i*x+j for j in range(k)],axis=0) for i in range(1,N+1)],axis=0)


def print_tensor(tensor, precision=4):
    tf.print(tf.strings.as_string(tensor,precision=precision))

def print_tensor_shape(tensor):
    tf.print("shape:",tensor.shape)

def reshape_query_set(layer_support_set_input, *args, **kwargs):
  return tf.reshape(layer_support_set_input,[-1,way*shot]+list(inp_shape))


In [7]:
# @title Assemble Model
def assemble_model_timeDist(nn_backbone, input_shape:tuple):
    layer_time_dist = layers.TimeDistributed(nn_backbone, name="Time_Distributed")

    # tuple of 2d input set shape containing one extra dimension
    input_shape_5d = (None,) + input_shape
    layer_support_set_input = layers.Input(input_shape_5d, name="Support_Set_Input")
    layer_query_set_input = layers.Input(input_shape, name="Query_Set_Input")

    layer_support_set_embeddings = layer_time_dist(layer_support_set_input)
    layer_prototypes = Lambda(produce_prototype,name="Prototypes")(layer_support_set_embeddings)

    layer_query_set_embedding = nn_backbone(layer_query_set_input)

    layer_output_prediction = Lambda(softmax_classification, name="Prediction")([layer_prototypes, layer_query_set_embedding])

    model = keras.Model(inputs=[layer_support_set_input, layer_query_set_input], outputs=layer_output_prediction, name="Atzori_DB2_Protonet")

    return model

"""
DESCRIPTION
    Instead of adding a time-distributed layer in front of the cnn backbone to process 2d sets of data
    it reshapes the support set into a 1d set. i.e. a (5,3) set of 15x12 images will be turned into a 5 x 3 = 15
    images of size 15x12 and will all be processed simultaneously.
    This will produces the 15 embeddings which will then be reshaped into the original 5x3 shape again
"""
def assemble_model_reshape(nn_backbone, input_shape:tuple, way:int, shot:int):
    input_shape_5d = (None,) + input_shape
    layer_support_set_input = layers.Input(input_shape_5d, name="Support_Set_Input")
    layer_query_set_input = layers.Input(input_shape, name="Query_Set_Input")

    # Concatenate the support set and query image into a 1d set of the standard input shape.
    # i.e. a (3,2) support set and 1 query image of size (15,12,1) will form a set of 2x3+1 = 7 images (15,12,1)
    layer_support_query_concat = tf.concat([tf.reshape(layer_support_set_input,[way*shot]+list(input_shape)), layer_query_set_input],axis=0)

    # We Produce Embeddings for all input data (support and query) by passing them all together through the network in feed forward fashion
    # Based on the previous example, given that a (15,12,1) image produces a (64,) long embedding, the whole
    # reshaped (7,15,12,1) set should produce a (7,64) set of embeddings
    layer_support_query_embeddings = nn_backbone(layer_support_query_concat)

    # Separate quary and support embeddings
    # Separate and reshape support set embeddings (6,64) -> (3,2,64)
    layer_support_set_embeddings = tf.reshape(layer_support_query_embeddings[:way*shot],[way,shot,layer_support_query_embeddings.shape[1]])
    layer_query_set_embedding = layer_support_query_embeddings[way*shot:]

    # Produce the prototypes of the support set
    # This should reduce the support set embeddings down to: (3,2,64) -> (3,64) by averaging the 2 embeddings of each of the 3 classes
    layer_prototypes = Lambda(produce_prototype)(layer_support_set_embeddings)

    # Make final prediction
    layer_prediction = Lambda(softmax_classification)([layer_prototypes, layer_query_set_embedding])

    # Assemble the final Model
    model = keras.Model(inputs = [layer_support_set_input, layer_query_set_input], outputs=layer_prediction)

    return model

def assemble_protonet_reshape_with_batch(nn_backbone, input_shape:tuple, way:int, shot:int):
    input_shape_4d = (1,) + input_shape
    input_shape_5d = (None,None,) + input_shape
    layer_support_set_input = layers.Input(input_shape_5d, name="Support_Set_Input")
    layer_query_set_input = layers.Input(input_shape_4d, name="Query_Set_Input")

    nn_backbone_timeDist = layers.TimeDistributed(nn_backbone)

    # Concatenate the support set and query image into a 1d set of the standard input shape.
    # i.e. a (3,2) support set and 1 query image of size (15,12,1) will form a set of 2x3+1 = 7 images (15,12,1)
    layer_support_query_concat = tf.concat([tf.reshape(layer_support_set_input,[-1,way*shot]+list(input_shape)), layer_query_set_input],axis=1)

    # We Produce Embeddings for all input data (support and query) by passing them all together through the network in feed forward fashion
    # Based on the previous example, given that a (15,12,1) image produces a (64,1) embedding, the whole
    # reshaped (7,15,12,1) set should produce a (7,64,1) set of embeddings
    layer_support_query_embeddings = nn_backbone_timeDist(layer_support_query_concat)

    # Separate query and support embeddings
    # Separate and reshape support set embeddings (6,64,1) -> (3,2,64,1)
    layer_support_set_embeddings = tf.reshape(layer_support_query_embeddings[:,:way * shot], [-1,way, shot, layer_support_query_embeddings.shape[-1]])
    layer_query_set_embedding = layer_support_query_embeddings[:,way*shot:]

    # Produce the prototypes of the support set
    # This should reduce the support set embeddings down to: (3,2,64,1) -> (3,64,1) by averaging the 2 embeddings of each of the 3 classes
    layer_prototypes = layers.Lambda(produce_prototype)(layer_support_set_embeddings)

    # Make final prediction
    layer_prediction = layers.Lambda(softmax_classification)([layer_prototypes, layer_query_set_embedding])

    # Assemble the final Model
    model = keras.Model(inputs = [layer_support_set_input, layer_query_set_input], outputs=layer_prediction, name="ProtoNet")

    return model


"""
DESCRIPTION
    Returns a set of fully connected layers, each with the number of neurons defined by a list parameter.
    Last layer is a single neuron layer providing a scalar output
    Each layer has a relu activation function except for the last which has a sigmoid (for providing a normalized output between 0.0 and 1.0)

PARAMETERS
    - neurons_per_layer = [128,64] would mean 2 layers of 128 and 64 neurons etc before the single neuron output
"""
def get_dense_layers(neurons_per_layer=[]):
    dense_layers = keras.Sequential()
    for i,neurons_number in enumerate(neurons_per_layer):
        dense_layers.add(layers.BatchNormalization())
        dense_layers.add(layers.Dense(units=neurons_number,activation='relu',name=f"dense_layer_{i+1}"))
    dense_layers.add(layers.Dense(units=1,activation='sigmoid', name=f"prediction_dense_layer"))

    return dense_layers


class SiameseNetwork(keras.Model):
    def __init__(self,cnn_backbone:keras.Model,f,inp_shape:tuple, dense_layers):
        super(SiameseNetwork, self).__init__()
        self.feature_extractor = cnn_backbone
        self.f = f
        self.inp_shape = inp_shape

        self.dense_layers = dense_layers

        return

    def call(self, input):
        x1,x2 = input

        embedding1 = self.feature_extractor(x1)
        embedding2 = self.feature_extractor(x2)

        embedding_dist = self.f([embedding1,embedding2])

        similarity_score = self.dense_layers(embedding_dist)

        return similarity_score


def assemble_siamNet_for_few_shot_infernce(model,inp_shape,N):
    feature_extractor = model.feature_extractor
    dense_layers = model.dense_layers
    f = model.f

    input_shape_4d = (1,) + inp_shape
    input_shape_5d = (None, None,) + inp_shape
    layer_support_set_input = layers.Input(input_shape_5d, name="Support_Set_Input")
    layer_query_set_input = layers.Input(input_shape_4d, name="Query_Set_Input")

    feature_extractor_timeDist = TimeDistributed(feature_extractor)
    feature_extractor_timeDist_support = TimeDistributed(feature_extractor_timeDist)

    layer_support_embeddings = feature_extractor_timeDist_support(layer_support_set_input)
    layer_query_embedding = feature_extractor_timeDist(layer_query_set_input)

    layer_support_set_prototypes = produce_prototype(layer_support_embeddings)
    layer_query_embedding_copied = tf.tile(layer_query_embedding, [1, N, 1])

    layer_dist_func = f([layer_support_set_prototypes, layer_query_embedding_copied])

    pred = TimeDistributed(dense_layers)(layer_dist_func)

    model_val = keras.Model(inputs=[layer_support_set_input, layer_query_set_input], outputs=pred)

    return model_val


# 3 - USEFUL FUNCTIONS

In [8]:
# @title helper functions

"""
DESCRIPTION
    Gets a list of keys (ie ['s1g1r3','s12g23r2','s12g40r1'] and returns the unique
    s,g,r values of all keys.
    In this example th unique subjects are 1,12, the unique gestures are 1,23,40
    and the unique reps are 1,2,3.
    Therefore the returning values are the following arrays:
    array([ 1, 12]), array([ 1, 23, 40]), array([1, 2, 3])
"""
def get_unique_sgr(keylist):
    pattern = r'\d+\.?\d*'
    sgr_list = np.array([[int(i) for i in re.findall(pattern, key)] for key in keylist])
    s = np.unique(sgr_list[:,0])
    g = np.unique(sgr_list[:,1])
    r = np.unique(sgr_list[:,2])

    return s,g,r

"""
Prints the keys of the support and query sets of a specific task in a formatted way
i.e. |  s1g1r3  s2g5r4  s12g7r3 |
     | s14g1r5  s5g5r1  s9g7r1  |

"""
def printKeys(keys):
    print()
    for j in range(len(keys[0])):
        print("| |", end='')
        for i in range(len(keys)):
            # Key takes up 8 cells of space
            print(keys[i][j],end="| |")
        print()


"""
DESCRIPTION
    For changing the formatting of the s,g,r values into the key String so that each int takes
    up 2 digits regardless of value
    ie : 's1g12r4' -> 's01g12r04'
    That way keys can also be easily sorted

PARAMETERS
    Key in old format ('s1g12r4')

OUTPUT
    Newly formatted key ('s01g12r04')
"""
def reformat_key(key):
    pattern = r'\d+\.?\d*'
    s,g,r = [int(i) for i in re.findall(pattern, key)]
    new_key = f"s{s:02d}g{g:02d}r{r:02d}"
    return new_key

"""
DESCRIPTION
    For getting a key based on given s,g,r values using the new format (2 digits regardless of value)
"""
def getKey(s,g,r):
    return f"s{s:02d}g{g:02d}r{r:02d}"

"""
DESCRIPTION
    Returns the name of the directory containing data of a certain database (db2 mainly) which are
    rms rectified (created offline) with a certain window size (in ms)
    i.e. 'db2_rms_100'
"""
def get_rms_rect_filename(db, win_size_ms):
    return f"db{db}_rms_{win_size_ms}.npz"

def get_rms_sub_filename(db, win_size_ms,new_freq):
    return f"db{db}_rms_{win_size_ms}_sub_{new_freq}.npz"

"""
DESCRIPTION
    For getting the full name of a configuration .json file (either for preprocessing, augmentation or training).
    depending on the mode the full filename is "config_{mode}_{filename}.json where filename usually includes info
    regarding the database if it is referring to a specific one

PARAMETERS
    mode : "preproc", "aug" or "train"
"""
def get_config_full_filename(mode, name):
    return f"config_{mode}_{name}.json"

"""
DESCRIPTION
    Loads and returns configuration data for either preprocessing, augmentation or training.
    Data is in the form of a jason file and the returning value is a dict

PARAMETERS
    mode : "preproc", "aug" or "train"
    filename : name of the file without the 'config_preproc' prefix of the '.json' postfix
"""
def get_config_from_json_file(mode, filename):
    full_filename = get_config_full_filename(mode, filename)
    if mode == "preproc":
        dir_path = DATA_CONFIG_PATH_PREPROC
    elif mode == "aug" :
        dir_path = DATA_CONFIG_PATH_AUG
    elif mode == "train" :
        return None
    else:
        return None

    with open(os.path.join(dir_path,full_filename)) as file:
        config = json.load(file)
        return config


"""
DESCRIPTION
    Returns the full filename of the .csv file where all the tasks for a given experiment exist
    It is something along the lines of ex{experiment}_{N}way_{k}shot.csv
    For the full path the FileInfoProvider in TaskGenerator takes care of it

PARAMETERS
    - mode : 'train', 'test' or 'val'
"""
def get_tasks_filename(ex,N,k, mode):
    return f'ex{ex}_{N}way_{k}shot_{mode}.csv'

def get_results_dir_fullpath(ex : str, N:int, k:int):
    return os.path.join(RESULTS_DIRECTORIES_DICT[ex],f'{N}_way_{k}_shot')

"""
    DESCRIPTION
    Determines the name of the model based on the number of existing models of the same type.
    i.e. it could be a protoNet with different backbone each time. It creates a file with the correct enumeration
    based on the number of existing models of that type.

    For example if there are already models 'model_protoNet_1.h5' and 'model_protoNet_2.h5' the file will be named
    'model_protoNet_3.h5' etc. If there are none the number will be set to '1'

    To provide mode info for the model and the training process in general a .txt file wll be provided
"""
def get_checkpoint_foldername(dir_path, model_name):
    name = f"model_{model_name}_1"
    while name in os.listdir(dir_path):
        num = int((name.split('.')[0]).split('_')[-1])
        name = f"model_{model_name}_{num + 1}"
    return name

"""
PARAMETERS
    criterion: 'latest', 'best_loss' or 'best_acc'
"""
def get_model_checkpoint_fullname(model_name, criterion):
    if criterion == 'latest':
        return model_name + '.h5'
    elif criterion == 'best_loss':
        return model_name + '_best_loss' + '.h5'
    elif criterion == 'best_acc':
        return model_name + '_best_acc' + '.h5'


"""
DESCRIPTION
    Takes as input a line in the form "7    	0.3781         	1.4724         	0.3410         	1.6121"
    and returns 7
"""
def get_line_starting_number(line):
    i=0
    while line[i].isdigit():
        i+=1

    return int(line[:i])

In [9]:
# @title plot functions

def plot_sEMG(emg, fs, channels=[1,2,3,4,5,6], title='', figure_name = 'Figure', export=False, filename=''):
  fig = plt.figure()
  fig.canvas.manager.set_window_title(figure_name)

  time = np.arange(len(emg))
  time = time/fs
  N = len(channels)

  for i in range(N):
    emg_channel = emg[:,channels[i]-1]
    max = np.max(np.abs(emg_channel))
    emg_norm = emg_channel/max
    offset = 2.5*(N-i-1)
    plt.plot(time,emg_norm+offset, label='channel {}'.format(i+1))

  lgd = plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
  plt.xlabel('time')
  plt.ylabel('channels')
  plt.title(title)
  plt.show()

"""
    DESCRIPTION
    Plots the spectrum of a 1-d channel of an semg signal
    If the all the channels of the signal are inserted it keeps the first one
    It shows the spectrum Amplitude for frequencies up to fs/2, either in multiples of pi or in Hz

    PARAMETERS
    x   : the semg signal channel
    fs  : sampling frequency
    title : plot title
    figure_name : name for the figure that will be initialized

    RETURNS
    Nothing
"""
def plotSpectrum(x,fs,showInMultiplesOfPi=False, title='Φάσμα emg σήματος', figure_name='Spectrum of sEMG Channel'):
  fig = plt.figure()
  fig.canvas.manager.set_window_title(figure_name)

  if(len(x.shape)>1):
    x = x[:,0]

  X = fft(x,5*len(x))
  L = len(X)
  L_lim = L//2
  if showInMultiplesOfPi:
    freq_lim = 1
    x_text = "Digital Frequency in multiples of pi"
  else:
    freq_lim = fs/2
    x_text = "Analog Frequency in Hz ([0,Fs/2])"

  df = freq_lim/(L_lim)

  freq = np.arange(0,freq_lim,df)
  magnitude = np.abs(X)[:L_lim]
  plt.plot(freq, magnitude,color='r',label='sEMG Spectrum')

  plt.xlabel(x_text)
  plt.title(title)
  plt.show()

  return

"""
DECRIPTION
  For plotting the values of a dictionary using a bar
  If keys have multiple values (same number of values for each key) they will be displayed using multiple colors (same for each category)

PARAMETERS
  data_dict : dictionary whose values are either lists (of same nuber of elements) or 1 value
  i.e. data_dict = {
        'a': [3, 4, 3],
        'b': [5, 7, 3],
        'c': [2, 2, 3]
      }
"""
def plotDictBar(data_dict:dict):
  total_bars = 100
  start = (27*49-10)*6
  ind = list(range(start,start+total_bars))
  keys = list(data_dict.keys())[start:start+total_bars]
  values = np.array(list(data_dict.values()))[start:start+total_bars]

  # Number of categories
  num_categories = values.shape[1]

  # Colors for each category
  colors = plt.cm.viridis(np.linspace(0, 1, num_categories))

  # Create a stacked bar chart
  fig, ax = plt.subplots()

  # Plot each category
  for i in range(num_categories):
    if i == 0:
      ax.bar(keys, values[:, i], color=colors[i], label=f'non-augmented')
    else:
      ax.bar(keys, values[:, i], bottom=np.sum(values[:, :i], axis=1), color=colors[i], label=f'augmented')

  plt.xlabel('Key')
  plt.ylabel('Value')

  # Add a legend
  plt.legend()

  # Show the plot
  plt.show()

  return


# 4 - DATA PROCESSING

In [10]:
# @title Preprocessing

def get_segmentation_indices(x: np.ndarray, window_size_ms: int, window_step_ms: int, fs):
    window_size = int((window_size_ms * fs) / 1000)
    window_step = int((window_step_ms * fs) / 1000)
    slice_start_indices = np.arange(0, len(x) - window_size + 1, window_step)
    return slice_start_indices


def subsample(x: np.ndarray, init_freq: float, new_freq: float):
    sub_factor = int(init_freq / new_freq)
    indices = np.arange(0, len(x), sub_factor)
    return np.take(x, indices=indices, axis=0)

def get_filter_coeffs(fc=1, fs=100, N=1):
    f = 2 * fc / fs
    b, a = scipy.signal.butter(N=N, Wn=f, btype='low')

    return b,a

def applyLPFilter(x, b ,a):
    x = np.abs(x)
    output = scipy.signal.filtfilt(b, a, x, axis=0, padtype='odd', padlen=3 * (max(len(b), len(a)) - 1))

    return output


def applyLPFilter2(emg, fc=1, fs=100, N=1):
    f_sos = scipy.signal.butter(N=N, Wn=2 * fc / fs, btype='low', output='sos')
    return scipy.signal.sosfilt(f_sos, emg, axis=0)

def minmax_norm(x):
    max_x = np.max(x)
    min_x = np.min(x)
    return (x-min_x) / (max_x-min_x)

"""
DESCRIPTION
    Performs μ-Law on a given emg signal x for a given value of μ
PARAMETERS
    - χ: The emg signal
    - μ: The value used n the formula
    - scaling_type: "all" for scaling all channels with the absolute maximum of the while signal
                    "each" for scaling with the maximum of each channel
"""
def muLaw_transform(x, mu = 2048, scaling_type = "all"):

    if scaling_type == "each":
        x = x/np.max(np.abs(x),axis=0)
    else:
        x= x/np.max(np.abs(x))

    return np.sign(x)*(np.log(1+mu*np.abs(x))/np.log(1+mu))


# Keeps only a certain amount of samples from each emg, the middle seconds_to_keep ones
def discard_early_and_late_gest_stages(x, seconds_to_keep, fs):
    num_samples_to_keep = int(seconds_to_keep*fs)
    # Half the length of samples to keep
    W = num_samples_to_keep // 2
    L = len(x)
    return x[max(L // 2 - W, 0):min(L // 2 + W, L)]


def apply_preprocessing(data_path, key_list, final_sample_subfix, total_subjects, config_dict:dict):
    print("Data processing...")
    print("...loading the data...")
    all_data = np.load(data_path)
    data = {key:all_data[key] for key in key_list}
    data_proc = {key:None for key in data}
    data_seg = {key:None for key in data}

    config_operations = config_dict['ops'].copy()
    config_params = config_dict['params'].copy()
    op_no_seg = [op for op in preprocess_operations if not op == "SEGMENT"]

    if config_operations["LOWPASS"] == True :
        b, a = get_filter_coeffs(**config_params["LOWPASS"])
        config_params["LOWPASS"] = {"b" : b, "a" : a}

    operations_params = [(preprocess_funcs[op], config_params[op]) for op in preprocess_operations if
                         config_operations[op] == True and op != "SEGMENT"]

    progress_bar = tqdm(total=total_subjects, desc="Preprocessing", unit=" reps")
    subjects_completed = 0
    t1 = time.time()
    for key, emg in data.items():
        # for testing purposes
        # for op in op_no_seg:
        # if key == "s12g34r04":
        #     print()
        for func, params in operations_params:
            emg = func(emg, **params)

        data_proc[key] = np.expand_dims(emg, -1)

        if key[3:] == final_sample_subfix:
            subjects_completed += 1
            progress_bar.set_postfix(subject=f'{key[:3]}')
            progress_bar.update(1)  # Update progress bar by 1
            # print(f"{key[:3]} ({subjects_completed}/{total_subjects})")
            t1 = time.time()

    if config_operations["SEGMENT"] == True:
        for key, emg in data_proc.items():
            data_seg[key] = get_segmentation_indices(emg, **config_params["SEGMENT"])
    progress_bar.close()

    return data_proc, data_seg






preprocess_operations = ["SUBSAMPLE", "DISCARD", "LOWPASS", "M-LAW", "MIN-MAX", "SEGMENT"]

preprocess_funcs = {
    "DISCARD"   :   discard_early_and_late_gest_stages,
    "SUBSAMPLE" :   subsample,
    "LOWPASS"   :   applyLPFilter,
    "M-LAW"     :   muLaw_transform,
    "MIN-MAX"   :   minmax_norm,
    "SEGMENT"   :   get_segmentation_indices
}


In [11]:
# @title RMS Rectification

def calculate_total_size(data_rms:dict):
    total_size = 0
    # Total size in bytes
    for key,emg in data_rms.items():
        total_size += emg.nbytes
    # size in giga bytes
    total_size/=(2**30)
    print(f"Total size: {total_size:.1f}Gb")

"""
DESCRIPTION
    For saving the dict of existing rms-rectified signals in the given path.
    Takes into account that the dict containing the rectified data, has all the possible
    keys for the specific database but not all data have been rectified, and so some values
    for specific keys are None.
    If no rectification has been done and no folder exists, it creates one based on the given name
"""
def save_rectified_gestures(data_rms: dict, full_path: str, filename:str):
    # Keeps only the non None values
    data_rms = {key: data_rms[key] for key in data_rms if data_rms[key] is not None}

    full_file_path = os.path.join(full_path,filename)
    np.savez(full_file_path, **data_rms)
    print(f"Rectified data saved at: '{full_file_path}'")
    calculate_total_size(data_rms)
    return

"""
DESCRIPTION
    Faster version for performing RMS Rectification on emg data
    Instead of calculating sum of squares over a given time window it calculates the squares and their
    cumulative sum beforehand and saves it in the emg_pad_csum array. That way emg_pad_csum[i] consists
    of the sum of all squares up to the i-th element in the original emg recording. Therefore, to calculate
    the sum of squares over a window (which covers the indices from i to j) all you need to do is calculate
    the difference between emg_pad_csum[j] and emg_pad_csum[i-1].

PARAMETERS
    x: emg recording to be rectified
    fs: sampling rate (2000 for DB2)
    win_size_ms: window size in milliseconds
"""
def rmsRect(x:np.ndarray, fs = 2000, win_size_ms=200):
    emg_rect = np.zeros(x.shape)
    W = int(win_size_ms*fs/1000)

    # npad: window_length/2 (used later for padding)
    npad = np.floor(W / 2).astype(int)
    win = int(W)

    # Symmetric padding with half the length of the window from each side
    # Thus ensuring the sliding window won't affect the total signal length
    # i.e. for x = [0,1,2,3,4,5,6,7] and W/2 == 3 symmetric padding should be
    #        [2,1,0,0,1,2,3,4,5,6,7,7,6,5]
    emg_pad = np.pad(x, ((npad, npad), (0, 0)), 'symmetric')

    # Square values of all cells
    emg_pad_squared = emg_pad**2
    # Cumulative sum along the time axis (where emg_pad_csum[i] = sum(emg_pad_squared[:i] for each channel)
    emg_pad_csum = np.cumsum(emg_pad_squared,axis=0)

    # emg[i] is replaced by the rms value of all the samples contained by the sliding window
    # centered in position i
    emg_pad_csum = np.pad(emg_pad_csum,((1,0),(0,0)) ,mode='constant', constant_values=0)

    emg_rect = np.sqrt((emg_pad_csum[win:-1]-emg_pad_csum[:-win-1])/win)
    return emg_rect


"""
DESCRIPTION

PARAMETERS
    db_dir_path:    full path of the directory where the data of the database in question exist
                    ie 'C:\\Users\\ΤΑΣΟΣ\\Desktop\\Σχολή\\Διπλωματική\\Δεδομένα\\processed\\db2'
"""
def apply_rms_rect(db: int, db_dir_path: str, fs: int, win_size_ms: int):
    rms_filename = get_rms_rect_filename(db, win_size_ms)  # ie 'db2_rms_100.npz'
    full_rms_file_path = os.path.join(db_dir_path, rms_filename)
    # rms_filename = rms_dir_name + '.npz'

    separated_data_filename = os.path.join(SEPARATED_DATA_PATH, f'db{db}.npz')
    data_sep_raw = np.load(separated_data_filename)

    already_rectified = 0

    # Case where the folder exists (and thus the rectification has either been completed or at least partially done
    if rms_filename in os.listdir(db_dir_path):
        # Checking whether all gestures have been rectified
        data_rms = np.load(full_rms_file_path)

        # If all the keys exist in the file, then rms rectification with that specific window size has already been doneand there is no need to redo
        if (set(data_rms.files) == set(data_sep_raw.files)):
            print("RMS Rectification with that window size already exists")
            return
        already_rectified = len(data_rms.files)
        remaining_keys = sorted(list(set(data_sep_raw.files) - set(data_rms.files)))
        # Copying all values of already rectified gestures to the corresponding keys in data_rms dict
        data_rms = {key: data_rms[key] for key in data_rms.files}
        # Initializing values for all keys of non-rectified gestures
        for key in remaining_keys:
            data_rms[key] = None

    else:
        remaining_keys = data_sep_raw.files
        # Initializing values for all keys to None
        data_rms = {key: None for key in remaining_keys}

    total_keys = len(remaining_keys) + already_rectified
    t_start = time.time()
    t1 = time.time()
    for i, key in enumerate(remaining_keys):
        emg = data_sep_raw[key]
        emg_rms = rmsRect(emg, win_size_ms=win_size_ms, fs=fs)

        data_rms[key] = np.copy(emg_rms)
        if (key[3:] == 'g49r06'):
            time_for_subject = time.time() - t1
            print(f"subject '{key[:3]}' ({already_rectified + i + 1}/{total_keys}) - {time_for_subject:.2f}s")
            t1 = time.time()

    print("total_time:", time.time() - t_start)
    save_rectified_gestures(data_rms, full_path=db_dir_path, filename=rms_filename)

    return



In [12]:
# @title Augmentation

def addGaussianNoise(emg:np.ndarray, snr_db:int = 25):
    # reduce dimension (time,channels,1) -> (time,channels)
    # i.e. (740,12,1) -> (740,12)
    emg = np.squeeze(emg)

    # provides the mean square value (power) along each channels
    # Should return a (1,channels) sized output where each element is the mean square value of each channel
    emg_mean_square = np.mean(emg**2, axis=0,keepdims=True)

    # SNR from db to linear value
    snr = 10 ** (snr_db/10)

    # Gaussian noise power (variance)
    noise_power = emg_mean_square/snr
    # stdev should be a (1,channels) array where each value is the stdev of the awgn for the respective channel
    stdev = np.sqrt(noise_power)

    # Gaussian Noise creation with 0 mean and the stdev we computed earlier
    awgn = np.random.normal(size=emg.shape, scale=stdev, loc=0.0)
    emg_n = emg + awgn

    return np.expand_dims(emg_n,-1)

def apply_augmentation(data, config_dict:dict, final_sample_subfix, total_subjects):
    print("Performing Data Augmentation...")
    data_aug = {key: None for key in data}
    ops = config_dict['ops']
    params = config_dict['params']

    progress_bar = tqdm(total=total_subjects, desc="Augmentation", unit=" reps")
    subjects_completed = 0
    t1 = time.time()
    for key,emg in data.items():

        for op in [op for op in augmentation_operations if ops[op] == True]:
            emg = augmentation_funcs[op](emg, **params[op])
        data_aug[key] = np.copy(emg)

        if key[3:] == final_sample_subfix :
            subjects_completed += 1
            progress_bar.set_postfix(subject=f'{key[:3]}')
            progress_bar.update(1)  # Update progress bar by 1
            # print(f"{key[:3]} ({subjects_completed}/{total_subjects}) : {time.time()-t1:.2f}s")
            t1 = time.time()

    progress_bar.close()
    print("\n")

    return data_aug



augmentation_operations = ["AWGN", "FLIP"]
augmentation_funcs = {
    "AWGN" : addGaussianNoise,
    "FLIP" : None
}

# 5 - CREATE RMS AND SUBSAMPLING FILES

In [ ]:
# @title Create RMS Rect
if __name__ == "__main__":

    db = 2
    fs = 2000
    win_size_ms = 150

    if db == 1:
        path = PROCESSED_DATA_PATH_DB1
    elif db == 2:
        path = RMS_DATA_PATH_DB2
    elif db == 5:
        path = PROCESSED_DATA_PATH_DB5
    else:
        exit(0)

    apply_rms_rect(db=db, db_dir_path=path, fs=fs, win_size_ms=win_size_ms)


In [ ]:
# @title Create Subsampling

if __name__ == "__main__":
    new_freq = 100
    for rms in [150,200]:
        unsampled_data_path = os.path.join(RMS_DATA_PATH_DB2,get_rms_rect_filename(2,rms))
        data = np.load(unsampled_data_path)
        subsampled_data = {}
        print(f"\n\nRMS Rectification for {rms}ms window size\n")
        t1 = time.time()
        for key,emg in data.items():
            emg_sub = subsample(emg,2000,new_freq=new_freq)
            subsampled_data[key] = emg_sub
            if key[3:] == "g49r06":
              print(f"{key[:3]}: {time.time() - t1}s")

        total_time = time.time() - t1
        print(f"total time for subsampling {len(data.items())} recordings: {total_time:.2f}s")

        new_path = os.path.join(RMS_DATA_PATH_DB2,get_rms_sub_filename(2,rms,new_freq))
        np.savez(new_path,**subsampled_data)

        print(f"rms {rms} done")



RMS Rectification for 150ms window size

s01: 0.5368850231170654s
s02: 1.0624680519104004s
s03: 1.6341922283172607s
s04: 2.3194808959960938s
s05: 2.8272173404693604s
s06: 3.3382062911987305s
s07: 4.001087665557861s
s08: 4.513712406158447s
s09: 5.096949577331543s
s10: 5.724949598312378s
s11: 6.416332006454468s
s12: 6.996959686279297s
s13: 7.469321250915527s
s14: 8.124029159545898s
s15: 8.664900779724121s
s16: 9.228147506713867s
s17: 9.979828357696533s
s18: 10.588258981704712s
s19: 11.125147342681885s
s20: 11.572941541671753s
s21: 12.041922092437744s
s22: 12.527730226516724s
s23: 13.075870513916016s
s24: 13.704082250595093s
s25: 14.239461183547974s
s26: 14.9082350730896s
s27: 15.456629991531372s
s28: 16.136059045791626s
s29: 16.67797040939331s
s30: 17.26772451400757s
s31: 17.97455668449402s
s32: 18.634765625s
s33: 19.180708646774292s
s34: 19.778010845184326s
s35: 20.473007202148438s
s36: 21.09112811088562s
s37: 21.784178495407104s
s38: 22.521363258361816s
s39: 23.28457283973694s
s40: 2

# 6 - DATA GENERATOR

In [13]:
# @title (S,G,R) Combinations for each experiment
sgr_domains = {
    "db2" : {
        "ex1" : {
            "s_domain" : {'train' : list(range(1,41)), 'test' : list(range(1,41))},
            "g_domain" : {'train' : list(range(1,50)), 'test' : list(range(1,50))},
            "r_domain" : {'train' : [1,3,4,6], 'test' : [2,5]}
        },
        "ex2" : {
            "s_domain" : {'train' : list(range(1,28)), 'val' : list(range(28,33)),'test' : list(range(33,41))},
            "g_domain" : {'train' : list(range(1,50)), 'val' : list(range(1,50)), 'test' : list(range(1,50))},
            "r_domain" : {'train' : list(range(1,7)),  'val' : list(range(1,7)),  'test' : list(range(1,7))}
        },
        "ex3" : {
            "s_domain" : {'train' : list(range(1,41)), 'val' : list(range(1,41)), 'test' : list(range(1,41))},
            "g_domain" : {'train' : list(range(1,35)), 'val' : list(range(35,41)),'test' : list(range(41,50))},
            "r_domain" : {'train' : list(range(1,7)),  'val' : list(range(1,7)),  'test' : list(range(1,7))}
        }
    },

    "db5" : {

    },

    "db1" : {

    }
}


In [14]:
# @title Class Task Generator

class FileInfoProvider:
    def __init__(self, db, rms, N, k, ex, mode,get_subsampled_data:bool=False):
        if db == 2:
            self.data_directory = RMS_DATA_PATH_DB2
            if get_subsampled_data == True:
                self.data_filepath = os.path.join(self.data_directory, get_rms_sub_filename(db, rms,100))
            else:
                self.data_filepath = os.path.join(self.data_directory, get_rms_rect_filename(db, rms))

        elif db == 1:
            self.data_directory = PROCESSED_DATA_PATH_DB1
            self.data_filepath = os.path.join(self.data_directory,'db1_raw.npz')
        elif db == 5:
            self.data_directory = PROCESSED_DATA_PATH_DB5
            self.data_filepath = os.path.join(self.data_directory,'db5_raw.npz')

        self.N = N
        self.k = k
        self.ex = ex
        self.mode = mode

    def setMode(self,mode):
        self.mode = mode
        return

    def getDataDirectoryPath(self):
        return self.data_directory

    def getDataFullPath(self):
        return self.data_filepath

    def getTasksFileFullPath(self):
        return os.path.join(TASKS_FILE_PATH_DB2,get_tasks_filename(ex=self.ex, N=self.N, k=self.k, mode=self.mode))


class TaskGenerator(utils.Sequence):
    def __init__(self, network_type:str, experiment:str, way:int, shot:int, mode:str,data_intake:str, database:int ,  preprocessing_config:dict,aug_enabled:bool, aug_config:dict, batch_size:int=1, batches: int = 1000, rms_win_size:int=200):
        self.experiment = experiment
        self.way = way
        self.shot = shot
        self.mode = mode
        self.data_intake = ''
        self.network_type = network_type
        self.task_generator = None
        self.db = database

        # Preprocessing and augmentation configurations
        self.preproc_config = preprocessing_config
        self.window_size = self.getWindowSize()
        self.rms_win_size = rms_win_size
        self.fileInfoProvider = FileInfoProvider(self.db, self.rms_win_size, N = self.way, k = self.shot, ex = self.experiment, mode=self.mode, get_subsampled_data=True)
        self.aug_enabled = aug_enabled
        if self.aug_enabled == True:
            self.aug_config = aug_config
            self.data_aug = {}

        # Only used if data_intake == 'csv'
        self.support_keys = []
        self.support_seg_start = []
        self.query_keys = []
        self.query_seg_start = []
        self.query_gest_indices = []

        # S,G,R Domains and acceptable keys
        self.s_domain = []
        self.g_domain = []
        self.r_domain = []
        self.s_r_pairs = []
        self.get_sgr_domains()
        self.key_list = self.get_all_acceptable_keys()

        print(f"~~~ {self.mode} loader ~~~".upper())
        self.get__data()
        # self.keyAppDict = self.get_key_app_dict()
        self.channels = self.getNumberOfChannels()

        self.batch_size = batch_size
        self.batches_per_epoch = batches

        # No need for any other data intake type other than 'generate'
        self.set_data_intake_type('generate')

        return

    def __getitem__(self, index):
        if self.network_type == "protoNet":
            all_keys = self.task_generator(index)
            support_batch, query_batch, labels_batch = self.get_task_data_based_on_keys(*all_keys)
        elif self.network_type == "siamNet":
            support_batch, query_batch, labels_batch = self.task_generator(index)

        return [support_batch, query_batch], labels_batch

    def __len__(self):
        return self.batches_per_epoch

    def get__data(self):
        """ As of now, apply_preprocessing is only used here. Therefore, total number of subjects and final
            subject key sub-fix are manually computed here as well """
        last_rep = self.r_domain[-1]
        last_gest = self.g_domain[-1]
        last_subfix = getKey(s=1,g=last_gest, r=last_rep)[3:]

        self.data, self.segments = apply_preprocessing(
            data_path=self.fileInfoProvider.getDataFullPath(),
            key_list=self.key_list, final_sample_subfix=last_subfix,
            total_subjects=len(self.s_domain),
            config_dict=self.preproc_config
        )
        if self.aug_enabled:
            self.data_aug = apply_augmentation(data=self.data, config_dict=self.aug_config, total_subjects=len(self.s_domain),final_sample_subfix=last_subfix)
        print()

        return

    def load_tasks_from_file(self):
        full_path = self.fileInfoProvider.getTasksFileFullPath()
        print('Loading tasks...')

        df = pd.read_csv(full_path)
        s_keys = df.iloc[:,:-3:2]
        s_seg = df.iloc[:,1:-3:2]
        q_keys = df.iloc[:,-3]
        q_seg = df.iloc[:,-2]
        q_label = df.iloc[:,-1]

        self.support_keys = np.reshape(s_keys.to_numpy(),[len(s_seg),self.way,self.shot])
        self.support_seg_start = np.reshape(s_seg.to_numpy(dtype=np.int32),[len(s_seg),self.way,self.shot])

        self.query_keys = q_keys.to_numpy()
        self.query_seg_start = q_seg.to_numpy()

        self.query_gest_indices = q_label.to_numpy()

        print("\n...tasks have been loaded.")

    def reshape_support_set(self,support_flattened):
        support_set = []
        for i in range(self.way):
            support_set.append(support_flattened[i*self.shot:(i+1)*self.shot])

        return support_set

    # Used for creating a dictionary keeping track of the number of times each key has been used
    # Used only for checking whether the task sampling process is correct
    def get_key_app_dict(self):
        keyAppDict = {}
        for key in self.data.keys():
            if self.aug_enabled:
                keyAppDict[key] = [0,0]
            else:
                keyAppDict[key] = [0]
        return keyAppDict


    def getNumberOfChannels(self):
        random_key = random.choice(list(self.data.keys()))
        random_sample = self.data[random_key]
        return random_sample.shape[1]

    # Returns the window size in number of time samples
    # i.e. if the sampling rate is 100Hz and the window size is meant to be 150ms that would return 15 samples
    def getWindowSize(self):
        w_ms = self.preproc_config['params']['SEGMENT']['window_size_ms']
        fs = self.preproc_config['params']['SEGMENT']['fs']
        return int((w_ms * fs) / 1000)

    def getKeys(self,*entries) -> list:
        return [getKey(s,g,r) for s,r,g in entries]

    def getKeys_in_order(self,*entries) -> list:
        return [getKey(s,g,r) for s,g,r in entries]

    def set_s_r_pairs(self) -> list:
        self.s_r_pairs = [(s,r) for s in self.s_domain for r in self.r_domain]

    """
    DESCRIPTION
        Depending on the experiment and the database used there are different values for the s,g,r fields
        These are all taken from the dictionary containing the configurations
    """
    def get_sgr_domains(self):
        exp_key = f'ex{self.experiment[0]}'
        db_key = f'db{self.db}'
        self.s_domain = sgr_domains[db_key][exp_key]["s_domain"][self.mode]
        self.g_domain = sgr_domains[db_key][exp_key]["g_domain"][self.mode]
        self.r_domain = sgr_domains[db_key][exp_key]["r_domain"][self.mode]
        self.set_s_r_pairs()

        return

    """
    DESCRIPTION
        Returns a list of all possible acceptable key combinations depending on the s,g,r domains

    """
    def get_all_acceptable_keys(self):
        sgr_comb_list = [(s,g,r) for s in self.s_domain for g in self.g_domain for r in self.r_domain]
        return self.getKeys_in_order(*sgr_comb_list)

    def plotKeyAppHist(self):
        plotDictBar(self.keyAppDict)


    def get_segment_of_semg(self, key, segment_start):
        # segment_start = random.choice(self.segments[key])
        indices = np.arange(segment_start, segment_start+self.window_size)
        if not self.aug_enabled:
            x = np.take(self.data[key],indices,axis=0)
            # self.keyAppDict[key][0] += 1
        else:
            #TODO - Might be more depending on the number of unique augmentation techniques used
            ind = np.random.choice([0,1]) # 0: non-aug, 1: aug
            x = np.take([self.data[key], self.data_aug[key]][ind],indices,axis=0)
            # self.keyAppDict[key][ind] += 1

        return x

    def set_batch_size(self, batch_size:int):
        self.batch_size = batch_size
        return

    """
        PARAMETERS
        data
    """
    def set_data_intake_type(self,data_intake):
        if self.data_intake == data_intake or data_intake not in ['csv','generate']:
            return
        self.data_intake = data_intake

        if self.network_type == "protoNet":
            if self.data_intake == "csv":
                self.load_tasks_from_file()
                self.task_generator = self.get_premade_keys
            elif self.data_intake == "generate":
                if self.experiment == '2a':
                    self.task_generator = self.generate_task_keys_2a
                elif self.experiment in ['1','2b','3']:
                    self.task_generator = self.generate_task_keys
        elif self.network_type == "siamNet":
            self.task_generator = self.generate_siamNet_task
        else:
            exit("invalid network type in TaskGenerator __init__. Should be between 'siamNet' and 'protoNet'")

        return

    def getMode(self):
        return self.mode
    def setMode(self,mode):
        if self.mode == mode:
            return
        self.mode = mode
        self.fileInfoProvider.setMode(self.mode)
        # self.s_r_pairs gets taken care of by self.get_sgr_domains
        self.get_sgr_domains()

        # In case tasks are premade and loaded from a csv file, they need to be reloaded since each csv file
        # depends on the mode and thus a different file contains the tak keys for that different mode
        if self.data_intake == "csv":
            # t1 = time.time()
            self.load_tasks_from_file()

        return

    def set_iterations_per_epoch(self,iterations):
        self.batches_per_epoch = iterations
        return

    def set_aug_enabled(self,aug_enabled):
        self.aug_enabled = aug_enabled


    def generate_task_keys(self, index):
        support_set_keys = []
        query_keys = []
        query_gest_indices = []
        support_segments_starting_indices = []
        query_segments_starting_indices = []

        for i in range(self.batch_size):
            support_set = []
            start_indices = []

            # Select N random gestures
            task_gestures = random.sample(self.g_domain, self.way)
            # Select 1 out of the N to be the query one and keep it index (which of the 5 it is)
            query_gesture_index, chosen_query_gest = random.choice(list(enumerate(task_gestures)))
            shot_list = [self.shot]*self.way
            shot_list[query_gesture_index] += 1

            support_pairs = [random.sample(self.s_r_pairs, shot_number) for shot_number in shot_list]
            for i,g in enumerate(task_gestures):
                sgr_list = [pair + (g,) for pair in support_pairs[i]]
                category_keys = self.getKeys(*sgr_list)
                support_set.append(category_keys)
                start_indices.append([random.choice(self.segments[key]) for key in category_keys])

            query_key = support_set[query_gesture_index].pop()
            query_seg_start_index = start_indices[query_gesture_index].pop()

            support_set_keys.append(support_set)
            support_segments_starting_indices.append(start_indices)
            query_keys.append(query_key)
            query_segments_starting_indices.append(query_seg_start_index)
            query_gest_indices.append(query_gesture_index)

        return support_set_keys, support_segments_starting_indices, query_keys, query_segments_starting_indices, query_gest_indices

    def generate_task_keys_2a(self, index):
        support_set_keys = []
        query_keys = []
        query_gest_indices = []
        support_segments_starting_indices = []
        query_segments_starting_indices = []

        for i in range(self.batch_size):
            support_set = []
            start_indices = []

            task_gestures = random.sample(self.g_domain, self.way)
            query_gesture_index, chosen_query_gest = random.choice(list(enumerate(task_gestures)))
            chosen_subject = random.choice(self.s_domain)
            shot_list = [self.shot] * self.way
            shot_list[query_gesture_index] += 1

            # Since each of the N gestures is taken from the same subject, the only variant in the (s,g,r)
            # combination of each key is the 'r'
            reps = [random.sample(self.r_domain, shot_number) for shot_number in shot_list]
            for i,g in enumerate(task_gestures):
                sgr_list = [(chosen_subject,rep,g) for rep in reps[i]]
                category_keys = self.getKeys(*sgr_list)
                support_set.append(category_keys)
                start_indices.append([random.choice(self.segments[key]) for key in category_keys])

            query_key = support_set[query_gesture_index].pop()
            query_seg_start_index = start_indices[query_gesture_index].pop()

            support_set_keys.append(support_set)
            support_segments_starting_indices.append(start_indices)
            query_keys.append(query_key)
            query_segments_starting_indices.append(query_seg_start_index)
            query_gest_indices.append(query_gesture_index)

        return support_set_keys, support_segments_starting_indices, query_keys, query_segments_starting_indices, query_gest_indices

    """
        Unlike generate_task_keys and generate_task_keys_2a it returns the data instead of their keys
    """
    def generate_siamNet_task(self, index):
        x1_batch = []
        x2_batch = []
        labels_batch = []

        for i in range(self.batch_size):
            # 0 for pair of different classes and 1 for pair of same class
            label = random.choice([0,1])
            if label == 0:
                gests = random.sample(self.g_domain, 2)
                sgr_list = [random.choice(self.s_r_pairs) + (g,) for g in gests]

            else:
                gest = random.choice(self.g_domain)
                s_r_pairs = random.sample(self.s_r_pairs,2)
                sgr_list = [s_r_pair + (gest,) for s_r_pair in s_r_pairs]

            key1,key2 = self.getKeys(*sgr_list)

            x1_batch.append(self.get_segment_of_semg(key1, random.choice(self.segments[key1])))
            x2_batch.append(self.get_segment_of_semg(key2, random.choice(self.segments[key2])))
            labels_batch.append(label)

        return np.array(x1_batch), np.array(x2_batch), np.array(labels_batch)





    def get_premade_keys(self,index):
        ind = np.arange(index*self.batch_size, (index+1)*self.batch_size)
        support_keys = self.support_keys[ind]
        query_keys = self.query_keys[ind]
        support_seg = self.support_seg_start[ind]
        query_seg = self.query_seg_start[ind]
        gest_ind = self.query_gest_indices[ind]

        return support_keys, support_seg, query_keys, query_seg, gest_ind

    def get_task_data_based_on_keys(self, support_keys, support_seg_starting_indices, query_keys, query_seg_starting_indices, query_gesture_indices):
        support_set_batch = []
        query_batch = []
        labels_batch = []

        # Each list should have length == self.batch_size
        for i in range(len(support_keys)):

            support_set = np.empty((self.way, self.shot, self.window_size, self.channels, 1))
            query_image = np.empty((1, self.channels, self.window_size, 1))

            for j, gest_key_list in enumerate(support_keys[i]):
                shots = [self.get_segment_of_semg(key,support_seg_starting_indices[i][j][k]) for k,key in enumerate(gest_key_list)]
                support_set[j] = np.array(shots)

            query_image = np.expand_dims(self.get_segment_of_semg(query_keys[i],query_seg_starting_indices[i]), axis=0)
            label = utils.to_categorical([query_gesture_indices[i]], num_classes=self.way)

            support_set_batch.append(support_set)
            query_batch.append(query_image)
            labels_batch.append(label[0])

        return np.array(support_set_batch), np.array(query_batch), np.array(labels_batch)


# 7 - GENERATE TASKS AND CONFIG FILES

In [ ]:
# @title Generate tasks

def create_csv_lines(support_keys,support_seg_start,query_keys,query_seg_start,query_gest_ind):
    batch_size,way,shot = np.array(support_keys).shape
    task_lines = np.empty([batch_size,2*(way*shot+1)+1],dtype='U9')
    s_keys_np = np.reshape(np.array(support_keys),[batch_size,way*shot])
    q_keys_np = np.expand_dims(np.array(query_keys),-1)
    s_q_keys = np.concatenate((s_keys_np,q_keys_np),axis=1)
    s_seg_np = np.reshape(np.array(support_seg_start),[batch_size,way*shot])
    q_seg_np = np.expand_dims(np.array(query_seg_start),-1)
    s_q_seg = np.concatenate((s_seg_np,q_seg_np),axis=1)

    task_lines[:,:-1:2] = s_q_keys
    task_lines[:,1:-1:2] = s_q_seg
    task_lines[:,-1] = query_gest_ind

    return task_lines

def save_tasks(task_lines,db,experiment,way,shot,mode):
    if db == 2:
      dirpath = TASKS_FILE_PATH_DB2
    elif db == 1:
      dirpath = TASKS_FILE_PATH_DB1
    elif db == 5:
      dirpath = TASKS_FILE_PATH_DB5

    full_path = os.path.join(dirpath,get_tasks_filename(ex=experiment,N=way,k=shot,mode=mode))

    with open(full_path,'w', newline='') as file:
        writer = csv.writer(file)
        for task in task_lines:
            writer.writerow(task)
    print(f"Saved at: '{full_path}'")

way = 5
shot = 5
ex = '1'
db = 2

num_batches = 10000
batch_size = 64

mode = 'train'
preproc_config = get_config_from_json_file(mode = 'preproc', filename = 'db2_no_lpf')
Gen = TaskGenerator(experiment='1', network_type='protoNet', way=way, shot=shot, mode=mode, data_intake='generate',database=db, preprocessing_config=preproc_config, aug_enabled=False, aug_config=None, rms_win_size=200, batch_size=batch_size, batches=num_batches, print_labels=False, print_labels_frequency=0)

t1 = time.time()
task_lines = np.empty([1+num_batches*batch_size,2*(way*shot+1)+1],dtype='U9')
headers = [el for i in range(way) for j in range(shot) for el in (f"c{i+1}_e{j+1}_key", f"c{i+1}_e{j+1}_seg")] + ['query_key','query_seg','label']
task_lines[0] = headers

print("Preparing tasks...")
for i in range(num_batches):
    keys = Gen.generate_task_keys(i)
    task_line =  create_csv_lines(*keys)
    task_lines[1 + i*batch_size : 1 + (i+1)*batch_size] = task_line
    if i%100 == 0:
        print(f"Batch {i}/{num_batches} : {time.time()-t1:.2f}")

print(f"Batch {i+1}/{num_batches} : {time.time() - t1:.2f}")

print("\n...tasks have been prepared")
print(f"total time for {num_batches*batch_size} ({way}way - {shot}shot) tasks: {time.time()-t1:.2f}")
save_tasks(task_lines,db=db, experiment=ex, way=way, shot=shot, mode=mode)


In [ ]:
# @title Add Config

preprocess_config = {
    "SUBSAMPLE" :   {"enable" : False,
                     "params" : {"init_freq" : 2000, "new_freq" : 100}},

    "DISCARD"   :   {"enable" : True ,
                     "params" : {"seconds_to_keep" : 5.0, "fs" : 100}},

    "LOWPASS"   :   {"enable" : True,
                     "params" : {"fc": 1, "fs": 100, "N": 1}},

    "MIN-MAX"   :   {"enable" : True,
                     "params" : {}},

    "M-LAW"     :   {"enable" : False,
                     "params" : {"mu":2048, "scaling_type":"all"}},

    "SEGMENT"   :   {"enable" : True ,
                     "params" : {"window_size_ms" : 150, "window_step_ms" : 60, "fs" : 100}}
}

preprocess_ops = {key : preprocess_config[key]["enable"] for key in preprocess_config.keys()}
preprocess_params = {key : preprocess_config[key]["params"] for key in preprocess_config.keys()}



# AUGMENTATION
augmentation_config = {
    "AWGN" : {"enable" : True ,
              "params" : {"snr_db" : 25}},

    "FLIP" : {"enable" : False ,
              "params" : None}
}

augmentation_ops = {key : augmentation_config[key]["enable"] for key in augmentation_config.keys()}
augmentation_params = {key : augmentation_config[key]["params"] for key in augmentation_config.keys()}

preproc_config_dict = {"ops" : preprocess_ops, "params" : preprocess_params}
aug_config_dict = {"ops": augmentation_ops, "params": augmentation_params}

def save_config(mode:str, filename:str):
    if mode == "preproc" :
        config_dict = {"ops" : preprocess_ops, "params" : preprocess_params}
        full_filename = get_config_full_filename(mode='preproc', name=filename)
        path = os.path.join(DATA_CONFIG_PATH_PREPROC, full_filename)

    elif mode == "aug" :
        config_dict = {"ops": augmentation_ops, "params": augmentation_params}
        full_filename = get_config_full_filename(mode='aug', name=filename)
        path = os.path.join(DATA_CONFIG_PATH_AUG,full_filename)

    elif mode == "train" :
        return

    with open(path, 'w') as file:
        json.dump(config_dict, file, indent=4)
#mode = 'aug'
#name = 'db2_awgn_snr25'
mode = 'preproc'
name = 'db2_discard_5.0_lpf_minmax_no_muLaw'
save_config(mode, name)

# 8 - TRAINING

In [15]:
# @title Custom callbacks

class IterationLoggingCallback(keras.callbacks.Callback):
    # def on_batch_end(self, batch, logs=None):
    #     if (batch % 100) == 0:
    #         # print(f"Batch {batch + 1}: loss = {logs.get('loss'):.2f}\n")
    #         print()
    def on_epoch_end(self, epoch, logs=None):
        super().on_epoch_end(epoch, logs)

class TrainingInfoCallback(keras.callbacks.Callback):
    # TODO - take into account changes regarding best_val_loss etc
    def __init__(self, file_path, model,  batch_size, model_filename, model_backbone_name, experiment, iterations_per_epoch, validation_steps, preprocessing_dict, aug_enabled, aug_dict, data_intake, rms, db):
        super(TrainingInfoCallback, self).__init__()
        self.file_path = file_path
        self.model = model
        self.batch_size = batch_size

        # 'model_protoNet_1.h5' -> 'model_protoNet_1'
        model_filename = model_filename.split('.')[0]
        self.json_filename = model_filename + "_training_info.json"
        self.txt_res_filename = model_filename + "_results.txt"
        self.json_fullpath = os.path.join(self.file_path, self.json_filename)
        self.txt_res_fullpath = os.path.join(self.file_path, self.txt_res_filename)

        self.model_backbone_name = model_backbone_name
        self.experiment = experiment
        self.iterations_per_epoch = iterations_per_epoch
        self.validation_steps = validation_steps
        self.preprocessing_dict = preprocessing_dict
        self.aug_enabled = aug_enabled
        self.aug_dict = aug_dict


        self.best_epoch_val_loss = 0
        self.best_epoch_val_acc = 0

        self.best_val_loss = float('inf')
        self.best_val_acc = 0.0

        self.best_loss_lr = self.get_lr()
        self.best_acc_lr = self.get_lr()

        self.load_results_if_exist()

        self.data_intake = data_intake
        self.rms         = rms
        self.db          = db

    def load_results_if_exist(self):
        if os.path.exists(self.json_fullpath):
            if os.path.exists(self.json_fullpath):
                with open(self.json_fullpath, 'r') as f:
                    training_info = json.load(f)

            self.best_epoch_val_loss = training_info["RESULTS"]["BEST_EPOCH_LOSS"]
            self.best_epoch_val_acc = training_info["RESULTS"]["BEST_EPOCH_ACC"]
            self.best_val_loss = training_info["RESULTS"]["BEST_VAL_LOSS"]
            self.best_val_acc = training_info["RESULTS"]["BEST_VAL_ACC"]
            self.best_loss_lr = training_info["RESULTS"]["BEST_EPOCH_LOSS_LEARNING_RATE"]
            self.best_acc_lr = training_info["RESULTS"]["BEST_EPOCH_ACC_LEARNING_RATE"]


    def round_results(self,logs):
        return {key: round(value, 4) for key, value in logs.items()}

    def get_lr(self):
        return float(self.model.optimizer.learning_rate.numpy())
    def update_json_log_file(self,epoch):
        if os.path.exists(self.json_fullpath):
            with open(self.json_fullpath, 'r') as f:
                training_info = json.load(f)
            # training_info["RESULTS"][f"epoch {epoch+1}"] = self.round_results(logs)
            training_info["RESULTS"]["TOTAL_EPOCHS"] = epoch+1
            training_info["RESULTS"]["LATEST_EPOCH_LEARNING_RATE"] = self.get_lr()

            training_info["RESULTS"]["BEST_VAL_LOSS"] = self.best_val_loss
            training_info["RESULTS"]["BEST_EPOCH_LOSS"] = self.best_epoch_val_loss
            training_info["RESULTS"]["BEST_EPOCH_LOSS_LEARNING_RATE"] = self.best_loss_lr

            training_info["RESULTS"]["BEST_VAL_ACC"] = self.best_val_acc
            training_info["RESULTS"]["BEST_EPOCH_ACC"] = self.best_epoch_val_acc
            training_info["RESULTS"]["BEST_EPOCH_ACC_LEARNING_RATE"] = self.best_acc_lr

        else:
            # Create a dictionary with training info
            training_info = {
                "MODEL" : {
                    "NAME" : self.model.name,
                    "BASE" : self.model_backbone_name
                },
                "PROCESSING" : {
                    "PREPROCESSING" : self.preprocessing_dict,
                    "AUG_ENABLED"   : self.aug_enabled,
                    "AUGMENTATION"  : self.aug_dict
                },
                "TRAINING_INFO" : {
                    "EXPERIMENT" : self.experiment,
                    "BATCH_SIZE" : self.batch_size,
                    "ITERATIONS_PER_EPOCH" : self.iterations_per_epoch,
                    "VALIDATION_STEPS" : self.validation_steps,
                    "OPTIMIZER" : self.model.optimizer._name,
                    "LOSS" : self.model.loss,
                    "METRICS" : self.model.metrics_names,
                },
                "RESULTS": {
                    "TOTAL_EPOCHS": epoch + 1,
                    "BEST_VAL_LOSS": self.best_val_loss,
                    "BEST_VAL_ACC": self.best_val_acc,
                    "BEST_EPOCH_LOSS": self.best_epoch_val_loss,
                    "BEST_EPOCH_ACC": self.best_epoch_val_acc,
                    "LATEST_EPOCH_LEARNING_RATE": self.get_lr(),
                    "BEST_EPOCH_LOSS_LEARNING_RATE": self.best_loss_lr,
                    "BEST_EPOCH_ACC_LEARNING_RATE": self.best_acc_lr
                },
                "DATA_GENERATOR" : {
                    "DATA_INTAKE" : self.data_intake,
                    "RMS" : self.rms,
                    "DB" : self.db
                }
            }
        # Save the dictionary as a JSON file
        with open(os.path.join(self.file_path,self.json_filename), 'w') as file:
            json.dump(training_info, file, indent=4)

    def write_txt_res_line(self,epoch,logs):
        train_acc = logs['train_accuracy']
        train_loss = logs['train_loss']
        val_acc = logs['val_accuracy']
        val_loss = logs['val_loss']

        # Format the result line
        result_line = f"{epoch + 1:<5}\t{train_acc:<15.4f}\t{train_loss:<15.4f}\t{val_acc:<15.4f}\t{val_loss:<15.4f}\n"

        # Append the result line to the file
        with open(self.txt_res_fullpath, 'a') as f:
            f.write(result_line)
    def update_txt_results_file(self,epoch,logs=None):
        if not os.path.exists(self.txt_res_fullpath):
            with open(self.txt_res_fullpath,'w') as res_file:
                column_names = f"{'Epoch':<5}\t{'Train Accuracy':<15}\t{'Train Loss':<15}\t{'Val Accuracy':<15}\t{'Val Loss':<15}\n"
                res_file.write(column_names)
                res_file.write("-" * 65 + "\n")

        self.write_txt_res_line(epoch,logs)

    # logs has the following form: {'train_loss':1.2, 'train_accuracy':0.6, 'val_loss':1.4, 'val_accuracy':0.5}
    def on_epoch_end(self, epoch, logs=None):
        self.update_json_log_file(epoch)
        self.update_txt_results_file(epoch,logs)

        return

class ReduceLrOnPlateauCustom(keras.callbacks.Callback):
    def __init__(self, model, reduction_factor, min_delta, best_val_loss , best_val_accuracy, patience, cooldown_patience, criterion="loss", epochs_without_improvement=0, cooldown_counter=0, min_lr=1e-5):
        super(ReduceLrOnPlateauCustom,self).__init__()

        # Model and values
        self.model = model
        self.min_delta = min_delta
        self.best_val_loss = best_val_loss
        self.best_val_accuracy = best_val_accuracy

        self.min_lr = min_lr
        self.reduction_factor = reduction_factor

        # Lr reduction patience
        self.patience = patience
        self.epochs_without_improvement = epochs_without_improvement

        # Cooldown patience
        self.cooldown_patience = cooldown_patience
        self.cooldown_counter = cooldown_counter

        # Criterion
        self.criterion = criterion

    def on_epoch_end(self, epoch, logs=None):
        current_lr = float(self.model.optimizer.learning_rate.numpy())
        val_loss = logs['val_loss']
        val_accuracy = logs['val_accuracy']
        min_lr_reached = (current_lr <= self.min_lr)
        if min_lr_reached == True:
            return True
        improvement_made = False

        # Improvement
        if self.criterion == "loss":
            if val_loss < self.best_val_loss - self.min_delta:
                print(f"new best loss {val_loss:.4f}")
                self.best_val_loss = val_loss
                self.epochs_without_improvement = 0
                improvement_made = True
        elif self.criterion == "accuracy":
            if val_accuracy > self.best_val_accuracy + self.min_delta:
                print(f"new best accuracy: {val_accuracy:.4f}")
                self.best_val_accuracy = val_accuracy
                self.epochs_without_improvement = 0
                improvement_made = True

        # If in cooldown mode no need to make any change yet
        if self.cooldown_counter > 0:
            print("You are in cooldown mode")
            self.cooldown_counter -= 1
            return min_lr_reached

        # No improvement
        if not improvement_made:
            self.epochs_without_improvement += 1
            # If Maximum epochs without improvement reached
            if self.epochs_without_improvement >= self.patience:
                # Calculate new lr
                # multiply current lr with the reduction factor
                current_lr = current_lr * self.reduction_factor

                if current_lr <= self.min_lr:
                    min_lr_reached = True
                    new_lr = self.min_lr
                    print(f"Minimum learning rate of {self.min_lr} reached")
                else:
                    new_lr = current_lr

                print(f"Reducing learning rate to {new_lr}")
                self.model.optimizer.learning_rate.assign(new_lr)

                # Reset epochs without improvement counter and enter cooldown
                self.epochs_without_improvement = 0
                print("Entering cooldown")
                self.cooldown_counter = self.cooldown_patience

        return min_lr_reached



class ReduceLrSteadilyCustom(keras.callbacks.Callback):
    def __init__(self,model,reduction_factor,patience, patience_counter=0, min_lr=1e-5):
        super(ReduceLrSteadilyCustom,self).__init__()
        self.model = model
        self.reduction_factor = reduction_factor
        self.patience = patience
        self.counter = patience_counter
        self.min_lr = min_lr

    def on_epoch_end(self, epoch, logs=None):
        self.counter += 1
        current_lr = float(self.model.optimizer.learning_rate.numpy())
        min_lr_reached = False

        if current_lr <= self.min_lr:
            print("Minimum lr already reached")
            return True
        # If reached enough epochs
        if self.counter >= self.patience:
            current_lr = current_lr*self.reduction_factor

            if current_lr <= self.min_lr:
                print(f"Minimum lr {self.min_lr:.6} reached")
                new_lr = self.min_lr
                min_lr_reached = True
            else:
                new_lr = current_lr

            print(f"New lr value: {new_lr:.6f}")
            self.model.optimizer.learning_rate.assign(new_lr)
            self.counter = 0

        return min_lr_reached


In [16]:
# @title Flags

LOAD_EXISTING_MODEL = False

SAVE_MODEL = False

# Callbacks
LR_SCHEDULER_ENABLED = True
CHECKPOINT_BEST_LOSS_ENABLED = False
CHECKPOINT_BEST_ACC_ENABLED = True
CHECKPOINT_LATEST_ENABLED = False
SAVE_TRAIN_STATS_ENABLED = False

EARLY_STOPPING_ENABLED = False


In [17]:
# @title Load training existing parameters

def keep_result_lines_until_best(filepath, epoch_to_keep_until):
    with open(filepath, 'r') as f_read:
        lines = f_read.readlines()
        lines_to_keep = lines[:2]
        for line in lines[2:]:
            if get_line_starting_number(line) <= epoch_to_keep_until: #TODO - Fix so that the whole number of the line is read
                lines_to_keep.append(line)

    with open(filepath, 'w') as f_write:
        for line in lines_to_keep:
            f_write.write(line)

    return

"""
PARAMETERS
    criterion : 'latest', 'best_loss' or 'best_acc'
"""
def get_training_config_from_json_file(json_filename,criterion):
    # training parameters
    global validation_steps
    global training_steps
    global batch_size

    # optimizer
    global optimizer
    global learning_rate
    global loss
    global metrics
    global starting_epoch

    # processing
    global preproc_config
    global aug_enabled
    global aug_config

    # data generator
    global data_intake
    global db
    global rms

    # accuracy and loss
    global best_val_loss
    global best_val_accuracy

    with open(json_filename) as f:
        info = json.load(f)
        # Processing
        preproc_config  = info["PROCESSING"]["PREPROCESSING"]
        aug_enabled     = info["PROCESSING"]["AUG_ENABLED"]
        aug_config      = info["PROCESSING"]["AUGMENTATION"]

        # Training Info
        training_steps   = info["TRAINING_INFO"]["ITERATIONS_PER_EPOCH"]
        validation_steps = info["TRAINING_INFO"]["VALIDATION_STEPS"]
        batch_size       = info["TRAINING_INFO"]["BATCH_SIZE"]
        optimizer_name   = info["TRAINING_INFO"]["OPTIMIZER"]
        loss             = info["TRAINING_INFO"]["LOSS"]
        metrics          = info["TRAINING_INFO"]["METRICS"]
        if 'loss' in metrics:
            metrics.remove('loss')

        # Results
        best_val_accuracy = info["RESULTS"]["BEST_VAL_ACC"]
        best_val_loss = info["RESULTS"]["BEST_VAL_LOSS"]
        if criterion == "latest":
            learning_rate = info["RESULTS"]["LATEST_EPOCH_LEARNING_RATE"]
            starting_epoch = info["RESULTS"]["TOTAL_EPOCHS"]
        elif criterion == "best_acc":
            learning_rate = info["RESULTS"]["BEST_EPOCH_ACC_LEARNING_RATE"]
            starting_epoch = info["RESULTS"]["BEST_EPOCH_ACC"]
        elif criterion == "best_loss":
            learning_rate = info["RESULTS"]["BEST_EPOCH_LOSS_LEARNING_RATE"]
            starting_epoch = info["RESULTS"]["BEST_EPOCH_LOSS"]
        else:
            exit("Wrong 'criterion' input in get_training_config_from_json_file()")

        # Data generator
        data_intake = info["DATA_GENERATOR"]["DATA_INTAKE"]
        db          = info["DATA_GENERATOR"]["DB"]
        rms         = info["DATA_GENERATOR"]["RMS"]

        if optimizer_name == 'Adam':
            optimizer = keras.optimizers.Adam(learning_rate)

    return



# 9 - PROTOTYPICAL NETWORK

In [ ]:
# @title Hardcoded training parameters (for new model)

validation_steps = 10000
training_steps = 1000
starting_epoch = 0
batch_size = 64
epochs = 25
win_size = 15
channels = 12
inp_shape = (win_size,channels,1)
cnn_backbone = AtzoriNetDB2_embedding_only_extra_layers_added(input_shape=inp_shape, add_dropout=False, add_regularizer=False, extra_layers=3)
learning_rate = 0.001
optimizer = keras.optimizers.Adam(learning_rate)
loss_function = 'categorical_crossentropy'
metrics = ['categorical_accuracy']

#input shape tuple
inp_shape_5d = (None,) + inp_shape

# DB and rms
db = 2
rms = 100

# experiment, way, shot
ex = '1'
N = 5
k = 5


# Results
best_val_loss = float('inf')
best_val_accuracy = 0.0

#cnn_backbone = AtzoriNetDB2_embedding_only()
cnn_backbone = AtzoriNetDB2_embedding_only_extra_layers_added(extra_layers=3)

resultsPath = get_results_dir_fullpath(ex, N, k)

# In case of LOAD_EXISTING_MODEL == True
model_name = 'model_ProtoNet_8'
criterion = 'best_loss'
#criterion = 'best_acc'
# criterion = 'latest'


if not LOAD_EXISTING_MODEL:
    print("Creating new model...\n")
    model = assemble_protonet_reshape_with_batch(cnn_backbone, inp_shape, way=N, shot=k)
    model.compile(loss=loss_function, optimizer=optimizer, metrics=metrics)
    model_foldername = get_checkpoint_foldername(resultsPath, model.name)
    print("Name:",model.name,'\n')

    #Results
    resultsPath = os.path.join(resultsPath, model_foldername)
    os.mkdir(resultsPath)
    checkpoint_latest_path = os.path.join(resultsPath, get_model_checkpoint_fullname(model_foldername, criterion='latest'))
    checkpoint_best_acc_path = os.path.join(resultsPath, get_model_checkpoint_fullname(model_foldername, criterion='best_acc'))
    checkpoint_best_loss_path = os.path.join(resultsPath, get_model_checkpoint_fullname(model_foldername, criterion='best_loss'))

    # Save initial state for all 3 models
    model.save(checkpoint_latest_path)
    model.save(checkpoint_best_loss_path)
    model.save(checkpoint_best_acc_path)

    print(f"...model saved at '{resultsPath}'")

else:
    print("Loading existing model...\n")
    print(f"Name: {model_name}\n")
    print(f"Criterion: {criterion}\n")

    model_foldername = model_name

    # Results paths
    resultsPath = os.path.join(resultsPath, model_foldername)
    checkpoint_latest_path = os.path.join(resultsPath, get_model_checkpoint_fullname(model_foldername, criterion='latest'))
    checkpoint_best_acc_path = os.path.join(resultsPath, get_model_checkpoint_fullname(model_foldername, criterion='best_acc'))
    checkpoint_best_loss_path = os.path.join(resultsPath, get_model_checkpoint_fullname(model_foldername, criterion='best_loss'))

    load_model_fullpath = {"latest":checkpoint_latest_path, "best_acc":checkpoint_best_acc_path, "best_loss":checkpoint_best_loss_path}[criterion]

    get_training_config_from_json_file(os.path.join(resultsPath,model_foldername + "_training_info.json"),criterion)
    keep_result_lines_until_best(filepath=os.path.join(resultsPath,model_foldername + "_results.txt"),epoch_to_keep_until=starting_epoch)
    print(f"...model loaded. Resuming training from epoch {starting_epoch}")

    model = keras.models.load_model(load_model_fullpath)
    model.compile(loss=loss_function, optimizer=optimizer, metrics=metrics)


preproc_config = get_config_from_json_file('preproc', 'db2_no_discard_lpf_minmax_muLaw')

aug_enabled = True
aug_config = get_config_from_json_file('aug', 'db2_awgn_snr25')
data_intake = 'generate'

data_loader = TaskGenerator(experiment=ex, way=N, shot=k, mode='train', data_intake=data_intake, database=db, preprocessing_config=preproc_config, aug_enabled=aug_enabled, aug_config=aug_config, rms_win_size=rms, batch_size=batch_size, batches=training_steps, print_labels=True, print_labels_frequency=5)

# Getting 1 output from train loader to test dimensions etc
[x,y], label = data_loader[0]


In [ ]:
# @title Training Callbacks

# Logger
iterationLoggingCallback = IterationLoggingCallback()

# Checkpoint
checkpointCallBack_val_loss = ModelCheckpoint(checkpoint_best_loss_path, save_best_only=True, monitor='val_loss', mode='min')
checkpointCallBack_val_loss.set_model(model)

checkpointCallBack_val_acc = ModelCheckpoint(checkpoint_best_acc_path, save_best_only=True, monitor='val_accuracy', mode='min')
checkpointCallBack_val_acc.set_model(model)

checkpointCallBack_latest = ModelCheckpoint(checkpoint_latest_path, save_freq='epoch')
checkpointCallBack_latest.set_model(model)

# Training info
trainingInfoCallback = TrainingInfoCallback(resultsPath, model, batch_size, model_foldername, cnn_backbone.name, ex, training_steps, validation_steps, preproc_config, aug_enabled, aug_config, data_intake, rms, db)

# LR Adjustment
reduction_factor = 0.5
patience = 2
cooldown_patience = 2
min_lr = 1e-4
min_delta = 0.001
lr_adjustment_callback = ReduceLrOnPlateauCustom(model=model, reduction_factor=reduction_factor, patience=patience,cooldown_patience=cooldown_patience,min_lr=min_lr, min_delta=min_delta, best_val_loss=best_val_loss)
#lr_adjustment_callback = ReduceLrSteadilyCustom(model=model, reduction_factor=reduction_factor,patience=patience,min_lr=min_lr)


early_stopping_mode_on = EARLY_STOPPING_ENABLED and (not LR_SCHEDULER_ENABLED)

In [ ]:
# @title Training Loop

for epoch_num in range(starting_epoch, starting_epoch+epochs):
    # training
    data_loader.setMode('train')
    data_loader.set_iterations_per_epoch(training_steps)
    data_loader.set_batch_size(batch_size)
    data_loader.set_aug_enabled(aug_enabled)
    print(f"\nEpoch {epoch_num+1:2d}/{starting_epoch+epochs}")

    # train for 1 epoch
    history = model.fit(data_loader, epochs=1, shuffle=False, callbacks=[iterationLoggingCallback])

    # validation
    if ex == '1':
      data_loader.setMode('test')
    else:
      data_loader.setMode('val')

    data_loader.set_iterations_per_epoch(validation_steps)
    data_loader.set_batch_size(1)
    data_loader.set_aug_enabled(False)
    print("Validation")
    val_loss, val_accuracy = model.evaluate(data_loader)
    logs = {"val_accuracy": val_accuracy, "val_loss": val_loss}

    # Model checkpoint: save model if it has the best performance
    if(val_loss < best_val_loss):
        best_val_loss = val_loss
        checkpointCallBack_val_loss.on_epoch_end(epoch_num, logs)

        trainingInfoCallback.best_val_loss = best_val_loss
        trainingInfoCallback.best_epoch_val_loss = epoch_num+1
        trainingInfoCallback.best_loss_lr = float(model.optimizer.learning_rate.numpy())

    if(val_accuracy > best_val_accuracy):
        best_val_accuracy = val_accuracy
        checkpointCallBack_val_acc.on_epoch_end(epoch_num,logs)

        trainingInfoCallback.best_val_acc = best_val_accuracy
        trainingInfoCallback.best_epoch_val_acc = epoch_num+1
        trainingInfoCallback.best_acc_lr = float(model.optimizer.learning_rate.numpy())

    checkpointCallBack_latest.on_epoch_end(epoch_num, logs)

    # Write results
    train_results = dict(**{"train_accuracy" : history.history['categorical_accuracy'][0], 'train_loss' : history.history['loss'][0]},**logs)
    trainingInfoCallback.on_epoch_end(epoch=epoch_num, logs=train_results)

    if LR_SCHEDULER_ENABLED:
        min_lr_reached = lr_adjustment_callback.on_epoch_end(epoch_num,logs)

        if min_lr_reached and EARLY_STOPPING_ENABLED:
            early_stopping_mode_on = True

    if early_stopping_mode_on:
        continue





Epoch  1/25
1000/1000 [==============================] - 198s 196ms/step - loss: 1.2480 - categorical_accuracy: 0.4621
Validation
10000/10000 [==============================] - 46s 5ms/step - loss: 1.1918 - categorical_accuracy: 0.4957
new best loss 1.1918

Epoch  2/25
1000/1000 [==============================] - 186s 186ms/step - loss: 1.1216 - categorical_accuracy: 0.5260
Validation
10000/10000 [==============================] - 46s 5ms/step - loss: 1.0888 - categorical_accuracy: 0.5454
new best loss 1.0888

Epoch  3/25
1000/1000 [==============================] - 186s 186ms/step - loss: 1.0502 - categorical_accuracy: 0.5607
Validation
10000/10000 [==============================] - 47s 5ms/step - loss: 1.0375 - categorical_accuracy: 0.5745
new best loss 1.0375

Epoch  4/25
1000/1000 [==============================] - 184s 184ms/step - loss: 1.0080 - categorical_accuracy: 0.5818
Validation
10000/10000 [==============================] - 46s 5ms/step - loss: 1.0049 - categorical_accura

In [ ]:
# @title Test

data_loader.setMode('test')
data_loader.set_iterations_per_epoch(20000)
data_loader.set_batch_size(1)
data_loader.set_aug_enabled(False)
print("Test")
test_loss, test_accuracy = model.evaluate(data_loader)
print('test_loss:',test_loss)
print('test_accuracy',test_accuracy)


Test
20000/20000 [==============================] - 94s 5ms/step - loss: 1.3469 - categorical_accuracy: 0.4794
test_loss: 1.346850872039795
test_accuracy 0.4793500006198883


In [ ]:
# @title GPU info

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


# 10 - SIAMESE NETWORK

In [ ]:
# @title Train SiamNet

def run_callbacks():
    logs = {"val_accuracy": val_accuracy, "val_loss": val_loss}
    early_stopping_mode_on = False

    if SAVE_MODEL == True:
        if CHECKPOINT_LATEST_ENABLED:    #TODO
            pass
        if CHECKPOINT_BEST_LOSS_ENABLED: #TODO
            pass
        if CHECKPOINT_BEST_ACC_ENABLED:  #TODO
            pass
        if SAVE_TRAIN_STATS_ENABLED:     #TODO
            train_results = dict(
                **{"train_accuracy": history.history['categorical_accuracy'][0], 'train_loss': history.history['loss'][0]},
                **logs)
            # trainingInfoCallback.on_epoch_end(epoch=epoch_num, logs=train_results)

    if LR_SCHEDULER_ENABLED:
        min_lr_reached = lr_adjustment_callback.on_epoch_end(epoch_num, logs)

        if min_lr_reached and EARLY_STOPPING_ENABLED:
            early_stopping_mode_on = True

    if EARLY_STOPPING_ENABLED and early_stopping_mode_on:
        pass

    return


def contrastive_loss(y_true, y_pred):
    # For the correct class (where y_true == 1), we want the similarity score to be close to 1
    positive_loss = y_true * K.square(1 - y_pred)

    # For the incorrect classes (where y_true == 0), we want the similarity score to be close to 0
    negative_loss = (1 - y_true) * K.square(y_pred)

    # Combine the positive and negative losses and average over all classes
    loss = K.mean(positive_loss + negative_loss)

    return loss

def evaluate_model(model:SiameseNetwork, data_loader, N):
    model_val = assemble_siamNet_for_few_shot_infernce(model=model, inp_shape=inp_shape, N=N)
    loss1 = 'categorical_crossentropy'
    loss2 = contrastive_loss
    model_val.compile(optimizer, loss=loss2, metrics=['categorical_accuracy'])
    print("Validation")
    val_loss, val_accuracy = model_val.evaluate(data_loader)

    return val_loss, val_accuracy


def evaluate_model2(model:SiameseNetwork, data_loader, validation_steps, N):
    # model.compile(optimizer=optimizer, loss=contrastive_loss, metrics=['categorical_accuracy'])
    inp_support_set = layers.Input(shape=(None,) + inp_shape)
    feature_extractor = model.feature_extractor
    feature_extractor_timeDist = keras.Model(inputs = inp_support_set, outputs=TimeDistributed(feature_extractor)(inp_support_set))

    dense_layers = model.dense_layers
    f = model.f

    correct_predictions = 0

    accuracy = 0.0

    progress_bar = tqdm(total=validation_steps, desc="Validation", unit=" iterations")

    for i in range(validation_steps):
        [support_set, query_image], labels = data_loader[i]
        # This is done because the support and the query image are taken in batches of 1
        support_set = support_set[0]
        query_image = query_image[0]

        # This produces a single L-long vector (where L is the chosen embedding shape for the feature extractor ie 64)
        query_embedding = feature_extractor(query_image)
        # We copy the embedding N times (as many as the classes of the suport set)
        # Input shape : (1,L) -> Output shape (N,L)
        query_embedding_copied = tf.tile(query_embedding, [N,1])

        # This produces a N,k set of L-long vectors
        support_embeddings = feature_extractor_timeDist(support_set)
        # Computes the prototypes of the classes that will go through the distance function
        # Input shape: (N,k,L) -> Output shape : (N,L)
        class_prototypes = produce_prototype(support_embeddings)

        # The distance function applied to the two (N,L) sets to produce a single output of N vectors of length L
        out_dist = f([query_embedding_copied, class_prototypes])

        # Final prediction after passing the output of the distance function through the dense layers
        pred = dense_layers(out_dist)

        if np.argmax(pred) == np.argmax(labels):
            correct_predictions += 1

        accuracy = correct_predictions/(i+1)
        progress_bar.set_postfix(accuracy=f"{accuracy:.4f}")
        progress_bar.update(1)  # Update progress bar by 1

    # progress_bar.close()

    return

training_steps = 2000
validation_steps = 5000
starting_epoch = 0
batch_size = 64
epochs = 80
win_size = 15
channels = 12
inp_shape = (win_size,channels,1)
learning_rate = 0.001
optimizer = keras.optimizers.Adam(learning_rate)
# loss_function = 'categorical_crossentropy'
loss_function = 'binary_crossentropy'
# metrics = ['categorical_accuracy']
metrics = ['binary_accuracy']


# DB and rms
db = 2
rms = 100

# experiment, way, shot
ex = '1'
N = 5
k = 5

# Results
best_val_loss = float('inf')
best_val_accuracy = 0.0

# LR Scheduler Parameters
reduction_factor = 0.5
patience = 25
cooldown_patience = 2
min_lr = 1e-4
min_delta = 0.001

cnn_backbone = AtzoriNetDB2_embedding_only_extra_layers_added(input_shape=inp_shape, extra_layers=3, add_dropout=False, add_regularizer=False)

resultsPath = os.path.join(RESULTS_DIRECTORIES_DICT[ex], get_results_dir_fullpath(ex, N, k))

print("Creating new model...\n")
# model = assemble_protonet_reshape_with_batch(cnn_backbone, inp_shape, way=N, shot=k)
model = SiameseNetwork(cnn_backbone=cnn_backbone, f=inner_product, inp_shape=inp_shape, dense_layers=get_dense_layers(neurons_per_layer=[]))
model.compile(optimizer=keras.optimizers.Adam(learning_rate), loss='binary_crossentropy', metrics=['binary_accuracy'])
model_foldername = get_checkpoint_foldername(resultsPath, model.name)

#Results
resultsPath = os.path.join(resultsPath, model_foldername)
if SAVE_MODEL == True:
    # os.mkdir(resultsPath)
    pass

checkpoint_latest_path = os.path.join(resultsPath, get_model_checkpoint_fullname(model_foldername, criterion='latest'))
checkpoint_best_acc_path = os.path.join(resultsPath, get_model_checkpoint_fullname(model_foldername, criterion='best_acc'))
checkpoint_best_loss_path = os.path.join(resultsPath, get_model_checkpoint_fullname(model_foldername, criterion='best_loss'))

# print(f"...model saved at '{resultsPath}'")


preproc_config = get_config_from_json_file('preproc', "db2_discard_3.5_lpf_minmax_no_muLaw")
aug_enabled = True
aug_config = get_config_from_json_file('aug', 'db2_awgn_snr25')
data_intake = 'generate'
# network_type = "protoNet"
network_type = "siamNet"


train_loader = TaskGenerator(network_type=network_type, experiment=ex, way=N, shot=k, mode='train', data_intake=data_intake, database=db, preprocessing_config=preproc_config, aug_enabled=aug_enabled, aug_config=aug_config, rms_win_size=rms, batch_size=batch_size, batches=training_steps)
test_loader = TaskGenerator(network_type='protoNet', experiment=ex, way=N, shot=k, mode='test', data_intake='generate', database=db, preprocessing_config=preproc_config, aug_enabled=False, aug_config=aug_config, rms_win_size=rms, batch_size=1, batches=validation_steps)

# Getting 1 output from train loader to test dimensions etc
[x,y], label = train_loader[0]
[x2,y2], label2 = test_loader[0]


# Custom Callbacks
# lr_adjustment_callback = ReduceLrOnPlateauCustom(model=model, criterion="loss", reduction_factor=reduction_factor, patience=patience,cooldown_patience=cooldown_patience,min_lr=min_lr, min_delta=min_delta, best_val_loss=best_val_loss, best_val_accuracy=best_val_accuracy)
lr_adjustment_callback = ReduceLrSteadilyCustom(model=model, reduction_factor=reduction_factor,patience=patience,min_lr=min_lr)
# checkpointCallBack_val_acc = ModelCheckpoint(checkpoint_best_acc_path, save_best_only=True, monitor='val_accuracy', mode='min')
# checkpointCallBack_val_acc.set_model(model)

for epoch_num in range(starting_epoch, starting_epoch+epochs):
    print(f"\nepoch {epoch_num + 1}/{starting_epoch + epochs}")
    print("Training")
    history = model.fit(train_loader,epochs=1)
    val_loss, val_accuracy = evaluate_model(model, test_loader, N)
    run_callbacks()

print()


Creating new model...

~~~ TRAIN LOADER ~~~
Data processing...
...loading the data...


Preprocessing:   0%|          | 0/40 [00:00<?, ? reps/s]

Performing Data Augmentation...


Augmentation:   0%|          | 0/40 [00:00<?, ? reps/s]




~~~ TEST LOADER ~~~
Data processing...
...loading the data...


Preprocessing:   0%|          | 0/40 [00:00<?, ? reps/s]



epoch 1/80
Training
2000/2000 [==============================] - 42s 20ms/step - loss: 0.6651 - binary_accuracy: 0.5851
Validation
5000/5000 [==============================] - 24s 5ms/step - loss: 0.2437 - categorical_accuracy: 0.3934

epoch 2/80
Training
2000/2000 [==============================] - 40s 20ms/step - loss: 0.6287 - binary_accuracy: 0.6372
Validation
5000/5000 [==============================] - 24s 5ms/step - loss: 0.2191 - categorical_accuracy: 0.4150

epoch 3/80
Training
2000/2000 [==============================] - 40s 20ms/step - loss: 0.6130 - binary_accuracy: 0.6568
Validation
5000/5000 [==============================] - 24s 5ms/step - loss: 0.1998 - categorical_accuracy: 0.4564

epoch 4/80
Training
2000/2000 [==============================] - 40s 20ms/step - loss: 0.6024 - binary_accuracy: 0.6667
Validation
5000/5000 [==============================] - 24s 5ms/step - loss: 0.1869 - categorical_accuracy: 0.4932

epoch 5/80
Training
2000/2000 [=======================